In [2]:
### TEMP 



# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
pd.set_option('display.max_colwidth', None)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

from matplotlib import pyplot as plt
import seaborn as sns

from google.cloud import bigquery
import datetime

import warnings
warnings.filterwarnings('ignore')



# function for safe queries (based on Kaggle SQL course)
def querytodf(query):
    dry_run_config = bigquery.QueryJobConfig(dry_run=True)

    # API request - dry run query to estimate costs
    dry_run_query_job = client.query(query, job_config=dry_run_config)
    print("This query will process {} bytes.".format(dry_run_query_job.total_bytes_processed))

    # Only run the query if it's less than 1 GB
    ONE_GB = 1000*1000*1000
    safe_config = bigquery.QueryJobConfig(maximum_bytes_billed=ONE_GB)

    # Set up the query (will only run if it's less than 1 GB)
    safe_query_job = client.query(query, job_config=safe_config)

    return safe_query_job.to_dataframe()

# get 'bls' dataset
client = bigquery.Client()
dataset_ref = client.dataset("bls", project="bigquery-public-data")
dataset = client.get_dataset(dataset_ref)



# check column info for CES series table
table_ref = dataset_ref.table('employment_hours_earnings_series')
table = client.get_table(table_ref)



# supersector codes
url = 'https://download.bls.gov/pub/time.series/ce/ce.supersector'
dfss = pd.read_csv(url, sep='\t')
# display(table.schema[1], '', dfss)

# industry codes
url = 'https://download.bls.gov/pub/time.series/ce/ce.industry'
dfi = pd.read_csv(url, sep='\t')
# display(table.schema[2], '', dfi)

def indlvl(num):
    return dfi[dfi.display_level == num].industry_name

# datatype codes
url = 'https://download.bls.gov/pub/time.series/ce/ce.datatype'
dft = pd.read_csv(url, sep='\t')
# display(table.schema[3], '', dft)

Using Kaggle's public dataset BigQuery integration.


In [3]:
display(plt.style.available)

plt.style.use('default')

# plt.style.use('bmh')
plt.style.use('fivethirtyeight')
# plt.style.use('seaborn-whitegrid')
# plt.style.use('seaborn-white')

plt.style.use('seaborn-notebook')

# sns.set_palette('tab20')
# sns.set_palette('Set2')

['Solarize_Light2',
 '_classic_test_patch',
 'bmh',
 'classic',
 'dark_background',
 'fast',
 'fivethirtyeight',
 'ggplot',
 'grayscale',
 'seaborn',
 'seaborn-bright',
 'seaborn-colorblind',
 'seaborn-dark',
 'seaborn-dark-palette',
 'seaborn-darkgrid',
 'seaborn-deep',
 'seaborn-muted',
 'seaborn-notebook',
 'seaborn-paper',
 'seaborn-pastel',
 'seaborn-poster',
 'seaborn-talk',
 'seaborn-ticks',
 'seaborn-white',
 'seaborn-whitegrid',
 'tableau-colorblind10']

# How did COVID-19 Impact the U.S. Labor Market?
# How is the U.S. Labor Market Recovering from COVID-19?

This notebook is inspired by some of the findings here:

https://www.kaggle.com/heisxiang/are-price-levels-increasing-faster-than-wages

namely that average wages rose dramatically during the onset of Covid-19

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
pd.set_option('display.max_colwidth', None)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

from matplotlib import pyplot as plt
import seaborn as sns

from google.cloud import bigquery
import datetime

import warnings
warnings.filterwarnings('ignore')

In [ ]:
# function for safe queries (based on Kaggle SQL course)
def querytodf(query):
    dry_run_config = bigquery.QueryJobConfig(dry_run=True)

    # API request - dry run query to estimate costs
    dry_run_query_job = client.query(query, job_config=dry_run_config)
    print("This query will process {} bytes.".format(dry_run_query_job.total_bytes_processed))

    # Only run the query if it's less than 1 GB
    ONE_GB = 1000*1000*1000
    safe_config = bigquery.QueryJobConfig(maximum_bytes_billed=ONE_GB)

    # Set up the query (will only run if it's less than 1 GB)
    safe_query_job = client.query(query, job_config=safe_config)

    return safe_query_job.to_dataframe()

# get 'bls' dataset
client = bigquery.Client()
dataset_ref = client.dataset("bls", project="bigquery-public-data")
dataset = client.get_dataset(dataset_ref)

# Examine Dataset to Build Queries
Explore the data to find what we want to query

Available tables:

In [ ]:
# list of tables
tables = client.list_tables(dataset_ref)
for table in tables: print(table.table_id)

Column info and first 5 rows of `employment_hours_earnings` table:

In [ ]:
# check column info for CES table
table_ref = dataset_ref.table('employment_hours_earnings')
table = client.get_table(table_ref)
display(table.schema)

# show first 5 rows
client.list_rows(table, max_results=5).to_dataframe()

Each set of time series data has a corresponding `series_id` and `series_title`. We need to find the relevant `series_id`'s that will help answer our question. To do this we'll examine the `employment_hours_earnings_series` table which has additional info about each `series_id`.

Column info and first 5 rows of `employment_hours_earnings_series` table:

In [ ]:
# check column info for CES series table
table_ref = dataset_ref.table('employment_hours_earnings_series')
table = client.get_table(table_ref)
display(table.schema)

# show first 5 rows
client.list_rows(table, max_results=5).to_dataframe()

We can see that each `series_id` corresponds with a number of codes describing its data. We'll explore the possible values of each code and cherry pick the ones relevant to our analysis. Files from https://download.bls.gov/pub/time.series/ce/

`supersector_code`:

In [ ]:
url = 'https://download.bls.gov/pub/time.series/ce/ce.supersector'
dfss = pd.read_csv(url, sep='\t')
display(table.schema[1], '', dfss)

`industry_code`:

In [ ]:
url = 'https://download.bls.gov/pub/time.series/ce/ce.industry'
dfi = pd.read_csv(url, sep='\t')
display(table.schema[2], '', dfi)

In [ ]:
def indlvl(num):
    return dfi[dfi.display_level == num].industry_name

# for lvl in dfi.display_level.unique():
#     display('Industry level ' + str(lvl), indlvl(lvl), '')

`data_type_code`:

In [ ]:
url = 'https://download.bls.gov/pub/time.series/ce/ce.datatype'
dft = pd.read_csv(url, sep='\t')
display(table.schema[3], '', dft)

`seasonal`:

In [ ]:
url = 'https://download.bls.gov/pub/time.series/ce/ce.seasonal'
df = pd.read_csv(url, sep='\t')
display(table.schema[4], '', df)

`footnote_codes`:

In [ ]:
url = 'https://download.bls.gov/pub/time.series/ce/ce.footnote'
df = pd.read_csv(url, sep='\t')
display(table.schema[6], '', df)

There is a lot of available data, the bls.gov website provides explanations here: https://www.bls.gov/web/empsit/cestn.htm

It looks like each industry has a corresponding level. We'll keep this in mind for later in our analysis. For now we'll take all sectors, all industries, seasonally adjusted data to eliminate noise, and the following data types:

Relevant data types for this analysis:
* 1	ALL EMPLOYEES, THOUSANDS
* 2	AVERAGE WEEKLY HOURS OF ALL EMPLOYEES
* 3	AVERAGE HOURLY EARNINGS OF ALL EMPLOYEES
* 6	PRODUCTION AND NONSUPERVISORY EMPLOYEES, THOUSANDS
* 7	AVERAGE WEEKLY HOURS OF PRODUCTION AND NONSUPERVISORY EMPLOYEES
* 8	AVERAGE HOURLY EARNINGS OF PRODUCTION AND NONSUPERVISORY EMPLOYEES
* 11	AVERAGE WEEKLY EARNINGS OF ALL EMPLOYEES
* 30	AVERAGE WEEKLY EARNINGS OF PRODUCTION AND NONSUPERVISORY EMPLOYEES
* 56	AGGREGATE WEEKLY HOURS OF ALL EMPLOYEES, THOUSANDS
* 57	AGGREGATE WEEKLY PAYROLLS OF ALL EMPLOYEES, THOUSANDS
* 81	AGGREGATE WEEKLY HOURS OF PRODUCTION AND NONSUPERVISORY EMPLOYEES, THOUSANDS
* 82	AGGREGATE WEEKLY PAYROLLS OF PRODUCTION AND NONSUPERVISORY EMPLOYEES, THOUSANDS

from `ce.text` located at https://download.bls.gov/pub/time.series/ce/:
<PRE> </PRE>
asdfasdfasdf

# Query + Prepare Data

Build query based on conditions determined above:

In [22]:
# build query
query = """ 
SELECT 
    ces.series_id, 
    ces_s.data_type_code, 
    ces_s.supersector_code, 
    ces_s.industry_code, 
    ces.year, 
    ces.period, 
    ces.date, 
    ces.value
FROM `bigquery-public-data.bls.employment_hours_earnings` ces
INNER JOIN `bigquery-public-data.bls.employment_hours_earnings_series` ces_s
    ON ces.series_id = ces_s.series_id
WHERE ces_s.data_type_code IN (1,2,3,6,7,8,11,30,56,57,81,82)
AND ces_s.seasonal = 'S'
AND EXTRACT(YEAR FROM ces.date) >= 1997
ORDER BY ces.series_id, ces.date
"""

# query data
data = querytodf(query)

This query will process 351343638 bytes.


In [23]:
### data cleaning + feature engineering

# replace codes with names
data['data_type_code'] = data['data_type_code'].replace(list(dft.data_type_code), 
                                                        list(dft.data_type_text))
data['supersector_code'] = data['supersector_code'].replace(list(dfss.supersector_code), 
                                                            list(dfss.supersector_name))
data['ind_lvl'] = data['industry_code'].replace(list(dfi.industry_code), 
                                                      list(dfi.display_level))
data['industry_code'] = data['industry_code'].replace(list(dfi.industry_code), 
                                                      list(dfi.industry_name))

data.rename(columns = {'data_type_code': 'data_type', 
                       'supersector_code': 'supersector', 
                       'industry_code': 'industry'}, 
            inplace=True)

# split 'data_type column' for easier slicing
dict1 = {'ALL EMPLOYEES, THOUSANDS': 'EMPLOYMENT OF ALL EMPLOYEES, THOUSANDS', 
         'PRODUCTION AND NONSUPERVISORY EMPLOYEES, THOUSANDS': 'EMPLOYMENT OF PRODUCTION AND NONSUPERVISORY EMPLOYEES, THOUSANDS'}
data['data_type'] = data.data_type.replace(dict1)
newcols = data.data_type.str.split(', ', expand=True)
# display(newcols)
newcols[[0,2]] = newcols[0].str.split(' OF ', expand=True)
# display(newcols)
newcols[0] = (newcols[0] + ', ' + newcols[1].fillna('')).str.rstrip(', ')
newcols.drop(1, axis=1, inplace=True)
data[['data_type', 'employee']] = newcols

# reorder columns
emp = data['employee']
data.drop('employee', axis=1, inplace=True)
data.insert(loc=2, column='employee', value=emp)

lvl = data['ind_lvl']
data.drop('ind_lvl', axis=1, inplace=True)
data.insert(loc=5, column='ind_lvl', value=lvl)

display(data)

,series_id,data_type,employee,supersector,industry,ind_lvl,year,period,date,value
0,CES0000000001,"EMPLOYMENT, THOUSANDS",ALL EMPLOYEES,Total nonfarm,Total nonfarm,0,1997,M01,1997-01-01,121363.0
1,CES0000000001,"EMPLOYMENT, THOUSANDS",ALL EMPLOYEES,Total nonfarm,Total nonfarm,0,1997,M02,1997-02-01,121675.0
2,CES0000000001,"EMPLOYMENT, THOUSANDS",ALL EMPLOYEES,Total nonfarm,Total nonfarm,0,1997,M03,1997-03-01,121990.0
3,CES0000000001,"EMPLOYMENT, THOUSANDS",ALL EMPLOYEES,Total nonfarm,Total nonfarm,0,1997,M04,1997-04-01,122286.0
4,CES0000000001,"EMPLOYMENT, THOUSANDS",ALL EMPLOYEES,Total nonfarm,Total nonfarm,0,1997,M05,1997-05-01,122546.0
...,...,...,...,...,...,...,...,...,...,...
1694502,CES9093299901,"EMPLOYMENT, THOUSANDS",ALL EMPLOYEES,Government,Other local government,5,2020,M11,2020-11-01,955.2
1694503,CES9093299901,"EMPLOYMENT, THOUSANDS",ALL EMPLOYEES,Government,Other local government,5,2020,M12,2020-12-01,946.0
1694504,CES9093299901,"EMPLOYMENT, THOUSANDS",ALL EMPLOYEES,Government,Other local government,5,2021,M01,2021-01-01,943.3
1694505,CES9093299901,"EMPLOYMENT, THOUSANDS",ALL EMPLOYEES,Government,Other local government,5,2021,M02,2021-02-01,946.2


In [52]:
# variables for slicing data

types = pd.Series(data.data_type.unique())
display('types:', types, '')

employees = data.employee.unique()
display('employees:', employees, '')

sectors = pd.Series(data.supersector.unique())
display('sectors:', sectors, '')

industries = data.industry.unique()
# display(industries)

# function for getting industry names by level
def indlvl(lvl):
    return dfi[dfi.display_level == lvl].industry_name

c19p = datetime.date(2020,1,1)
c19a = datetime.date(2020,2,1)
c19b = datetime.date(2020,4,1)

cdate19 = data.year == 2019
cdatep = data.date == c19p
cdatea = data.date == c19a
cdateb = data.date == c19b

'types:'

0                   EMPLOYMENT, THOUSANDS
1                    AVERAGE WEEKLY HOURS
2                 AVERAGE HOURLY EARNINGS
3                 AVERAGE WEEKLY EARNINGS
4       AGGREGATE WEEKLY HOURS, THOUSANDS
5    AGGREGATE WEEKLY PAYROLLS, THOUSANDS
dtype: object

''

'employees:'

array(['ALL EMPLOYEES', 'PRODUCTION AND NONSUPERVISORY EMPLOYEES'],
      dtype=object)

''

'sectors:'

0                            Total nonfarm
1                            Total private
2                          Goods-producing
3                        Service-providing
4                Private service-providing
5                       Mining and logging
6                             Construction
7                            Manufacturing
8                            Durable Goods
9                         Nondurable Goods
10    Trade, transportation, and utilities
11                         Wholesale trade
12                            Retail trade
13          Transportation and warehousing
14                               Utilities
15                             Information
16                    Financial activities
17      Professional and business services
18           Education and health services
19                 Leisure and hospitality
20                          Other services
21                              Government
dtype: object

''

# EDA

time-series simple viz: e,h,e (all vs ns)

pre-covid:
sup vs ns employees (ratio): emp, hours, earnings
avg hours/earnings

Industry size: emp, hours, earnings
avg hours/earnings
which ind has highest sup-ns ratio?
which ind has largest wage gap between sup-ns? hours gap?

pre-covid trends:
overall growth rate
by industry: highest/lowest growth

In [58]:
data.isna().sum()

series_id      0
data_type      0
employee       0
supersector    0
industry       0
ind_lvl        0
year           0
period         0
date           0
value          0
dtype: int64

In [ ]:
data.info()

In [55]:
# categorical variables
cat = data.columns[[0,1,2,3,4,6]]
cat

Index(['series_id', 'data_type', 'employee', 'supersector', 'industry',
       'year'],
      dtype='object')

In [56]:
for c in cat:
    display(c, data[c].nunique(), '')

'series_id'

6965

''

'data_type'

6

...

864

''

'year'

25

''

In [57]:
display(
    'min dates:', data.groupby('series_id').date.min().value_counts(), '',
    'max dates:', data.groupby('series_id').date.max().value_counts(), '', 
    'series per data_type:', data.groupby('data_type').series_id.nunique().sort_values(), '', 
    'series per supersector:', data.groupby('supersector').series_id.nunique().sort_values(), '',
    'series per industry:', data.groupby('industry').series_id.nunique().sort_values(), '',
)

'min dates:'

1997-01-01    3929
2006-03-01    3010
2003-01-01      24
2001-01-01       2
Name: date, dtype: int64

''

'max dates:'

2021-03-01    6349
2021-04-01     616
Name: date, dtype: int64

...

supersector
Total nonfarm                             1
Service-providing                         1
Trade, transportation, and utilities     12
Total private                            12
Goods-producing                          12
Manufacturing                            12
Private service-providing                12
Government                               21
Utilities                                87
Mining and logging                      143
Information                             194
Transportation and warehousing          326
Leisure and hospitality                 423
Nondurable Goods                        442
Construction                            472
Other services                          487
Education and health services           571
Financial activities                    584
Wholesale trade                         585
Retail trade                            742
Professional and business services      900
Durable Goods                           926
Name: series_id, dty

''

'series per industry:'

industry
AC, refrigeration, and forced air heating                     1
Government                                                    1
Rail transportation                                           1
Railroad rolling stock and other transportation equipment     1
Glass containers and products made of purchased glass         1
                                                             ..
Chemicals                                                    24
Motor vehicles and parts                                     24
Motor vehicles                                               24
Paper and paper products                                     24
Durable goods                                                24
Name: series_id, Length: 864, dtype: int64

''

In [7]:
# industries per supersector
data.groupby('supersector').industry.value_counts()

supersector      industry                                      
Construction     Construction                                      2954
                 Construction of buildings                         2943
                 Heavy and civil engineering construction          2943
                 Nonresidential building                           2943
                 Residential building                              2943
                                                                   ... 
Wholesale trade  Home furnishings                                   291
                 Industrial paper                                   291
                 Printing and writing paper and office supplies     291
                 Wine and spirits                                   291
                 Women's and children's clothing                    291
Name: industry, Length: 871, dtype: int64

In [ ]:
# check for missing months
s = data.groupby('series_id').period.value_counts().rename('period_counts').to_frame().reset_index()
display(s)
s2 = s.groupby('series_id').period_counts.max() - s.groupby('series_id').period_counts.min()
s2.value_counts() # max should be 1

In [9]:
# eda for wage levels pre-covid

AWE = types[3]
display(AWE)
ind = indlvl(3)
display(ind)

df = data[(data.data_type == AWE)
          & (data.industry.isin(ind))
         ]
# display(df)

# this should always be 1:
df.groupby(['supersector','industry', 'employee']).series_id.nunique().value_counts()

'AVERAGE WEEKLY EARNINGS'

7                                       Mining
76                               Durable goods
219                           Nondurable goods
305                            Wholesale trade
367                               Retail trade
452             Transportation and warehousing
493                                  Utilities
530                      Finance and insurance
569         Real estate and rental and leasing
595        Professional and technical services
645    Management of companies and enterprises
648          Administrative and waste services
694                       Educational services
707          Health care and social assistance
762        Arts, entertainment, and recreation
788            Accommodation and food services
859                                    Federal
865                           State government
871                           Local government
Name: industry_name, dtype: object

1    30
Name: series_id, dtype: int64

In [15]:
# industries by pre-covid wages


AWE = types[3]
display(AWE)
ind = indlvl(3)
display(ind)

df = data[(data.data_type == AWE)
          & (data.industry.isin(ind))
          & (data.date == c19p)]

display(df.groupby(['supersector','industry', 'employee']).value.mean().sort_values(ascending=False).reset_index())

'AVERAGE WEEKLY EARNINGS'

7                                       Mining
76                               Durable goods
219                           Nondurable goods
305                            Wholesale trade
367                               Retail trade
452             Transportation and warehousing
493                                  Utilities
530                      Finance and insurance
569         Real estate and rental and leasing
595        Professional and technical services
645    Management of companies and enterprises
648          Administrative and waste services
694                       Educational services
707          Health care and social assistance
762        Arts, entertainment, and recreation
788            Accommodation and food services
859                                    Federal
865                           State government
871                           Local government
Name: industry_name, dtype: object

,supersector,industry,employee,value
0,Utilities,Utilities,ALL EMPLOYEES,1800.73
1,Professional and business services,Management of companies and enterprises,ALL EMPLOYEES,1623.82
2,Mining and logging,Mining,ALL EMPLOYEES,1617.46
3,Utilities,Utilities,PRODUCTION AND NONSUPERVISORY EMPLOYEES,1612.28
4,Professional and business services,Professional and technical services,ALL EMPLOYEES,1596.18
5,Mining and logging,Mining,PRODUCTION AND NONSUPERVISORY EMPLOYEES,1438.08
6,Professional and business services,Professional and technical services,PRODUCTION AND NONSUPERVISORY EMPLOYEES,1336.97
7,Wholesale trade,Durable goods,ALL EMPLOYEES,1259.96
8,Wholesale trade,Wholesale trade,ALL EMPLOYEES,1231.12
9,Durable Goods,Durable goods,ALL EMPLOYEES,1211.46


In [53]:
# final DF

# columns = ['series_id', 'data_type', 'supersector', 'industry', 'ind_lvl', 'employee', 'Avg2019', 'Jan2020', 'Feb2020', 'Apr2020', 'change', '% change']

In [49]:
# transform queried data into final form

cols = ['series_id', 
        'data_type', 
        'supersector', 
        'industry', 
        'ind_lvl', 
        'employee']

df0 = data[cdate19].groupby(cols).value.mean().rename('Avg2019').reset_index()
df0['Jan2020'] = data[cdatep].groupby(cols).value.mean().values
df0['Feb2020'] = data[cdatea].groupby(cols).value.mean().values
df0['Apr2020'] = data[cdateb].groupby(cols).value.mean().values
df0['change'] = df0['Apr2020'] - df0['Feb2020']
df0['% change'] = (df0['Apr2020'] - df0['Feb2020']) / df0['Feb2020']

display(df0)

,series_id,data_type,supersector,industry,ind_lvl,employee,Avg2019,Jan2020,Feb2020,Apr2020,change,% change
0,CES0000000001,"EMPLOYMENT, THOUSANDS",Total nonfarm,Total nonfarm,0,ALL EMPLOYEES,150900.083333,152234.00,152523.00,130161.00,-22362.00,-0.146614
1,CES0500000001,"EMPLOYMENT, THOUSANDS",Total private,Total private,1,ALL EMPLOYEES,128290.416667,129445.00,129688.00,108335.00,-21353.00,-0.164649
2,CES0500000002,AVERAGE WEEKLY HOURS,Total private,Total private,1,ALL EMPLOYEES,34.391667,34.30,34.40,34.20,-0.20,-0.005814
3,CES0500000003,AVERAGE HOURLY EARNINGS,Total private,Total private,1,ALL EMPLOYEES,27.995833,28.43,28.51,30.07,1.56,0.054718
4,CES0500000006,"EMPLOYMENT, THOUSANDS",Total private,Total private,1,PRODUCTION AND NONSUPERVISORY EMPLOYEES,105574.500000,106444.00,106651.00,87038.00,-19613.00,-0.183899
...,...,...,...,...,...,...,...,...,...,...,...,...
6960,CES9093222101,"EMPLOYMENT, THOUSANDS",Government,Local government utilities,5,ALL EMPLOYEES,246.875000,248.20,248.20,247.10,-1.10,-0.004432
6961,CES9093248001,"EMPLOYMENT, THOUSANDS",Government,Local government transportation,5,ALL EMPLOYEES,290.283333,295.50,294.90,293.90,-1.00,-0.003391
6962,CES9093262201,"EMPLOYMENT, THOUSANDS",Government,Local hospitals,5,ALL EMPLOYEES,682.191667,689.20,690.40,677.40,-13.00,-0.018830
6963,CES9093292001,"EMPLOYMENT, THOUSANDS",Government,Local government general administration,5,ALL EMPLOYEES,4255.641667,4286.70,4295.70,4107.50,-188.20,-0.043811


# How did COVID-19 Impact the U.S. Labor Market?

### Outline
Definitions from https://www.bls.gov/web/empsit/cestn.htm#section5:

Basic data types:
* Total employees
* Total weekly payroll
* Total weekly hours

Derived data types:
* Average weekly hours = total weekly hours / total employees
* Average hourly earnings = total weekly payroll / total weekly hours
* Average weekly earnings = average hourly earnings * average weekly hours (total weekly payroll / total employees)

Hypothesis:
* Many of the people who lost their jobs during COVID-19 were relatively lower wage earners, thus leaving relatively higher wage earners in the payroll data, driving average wages higher.

Method: 

Compare pre vs during COVID-19 numbers for each of the datatypes

**Total private sector**
* AHE vs AWE
* Basic Data
    * All employees vs non-supervisory
    * abs vs pct
* AWH: total hours vs total employees
* AHE: payroll vs total hours
* AWE: AHE vs AWH
* AWE: payroll vs total employees

**by industry:**
* Basic data: change by industry (tiered by average earnings pre-covid)
* Hours change?

### Total Private Sector

* Basic Data
    * All employees vs non-supervisory
    * abs vs pct

In [ ]:
ind = indlvl(1).values[0]
typ = types[4]
c19a = datetime.date(2020,2,1)
c19b = datetime.date(2020,4,1)

cdate1 = data.date >= datetime.date(2019,7,1)
cdate2 = data.date <= datetime.date(2020,12,1)
ctype = data.data_type == typ
cind = data.industry == ind

dfog = data[cdate1&cdate2&ctype&cind]
df = dfog.groupby(['data_type', 'employee', 'date']).value.sum().reset_index()

# difference between start and worst of drop
df0 = df[df.employee == df.employee.unique()[0]]
start0 = df0[df0.date == c19a].value.mean()
end0 = df0[df0.date == c19b].value.mean()
diff0 = end0 - start0

df1 = df[df.employee == df.employee.unique()[1]]
start1 = df1[df1.date == c19a].value.mean()
end1 = df1[df1.date == c19b].value.mean()
diff1 = end1 - start1

plt.figure(figsize=(6,3))
sns.lineplot(data=df, x='date', y='value', hue='employee',)
plt.title('COVID-19 Impact: U.S. ' + ind + ', ' + typ, y=1.3)
plt.legend(loc='lower left', bbox_to_anchor=(-0.2,1.05))

plt.axvspan(datetime.date(2020,2,1), datetime.date(2020,4,1), 
            color='red', alpha=0.2)

plt.text(datetime.date(2020,4,1), start0+diff0/2, 
         str('{:,}'.format(round(diff0)))+'\n({:.2%})'.format(diff0/start0),
         color = sns.color_palette()[0],
         weight='bold')
plt.text(datetime.date(2020,4,1), start1+diff1/2, 
         str('{:,}'.format(round(diff1)))+'\n({:.2%})'.format(diff1/start1),
         color = sns.color_palette()[1],
         weight='bold')

plt.xticks(rotation=20); 

Average weekly hours = total weekly hours / total employees

AWH: total hours vs total employees

### by industry

In [ ]:
indlvl(3)

In [ ]:
df2 = data[ctype
           & data.industry.isin(indlvl(4).values)
           & data.date.isin([c19a, c19b])]
df2 = df2.groupby(['industry', 'data_type', 'employee', 'date']).value.sum().reset_index()
df2 = df2.pivot(index=['industry', 'data_type', 'employee'], columns='date', values='value').reset_index()
df2.columns.name = None
df2['diff'] = df2[df2.columns[4]] - df2[df2.columns[3]]
df2['% change'] = (df2[df2.columns[4]] - df2[df2.columns[3]])/df2[df2.columns[3]]
df2

In [ ]:
order = df2[df2.employee == 'ALL EMPLOYEES'].groupby('industry')['% change'].min().sort_values().head(15)
plt.figure(figsize=(4,6))
sns.barplot(data=df2, y='industry', x='% change', hue='employee', order=order.index)
plt.legend(loc='upper left', bbox_to_anchor=(-0.5,-0.1))

order = df2[df2.employee == 'ALL EMPLOYEES'].groupby('industry')['% change'].min().sort_values().tail(15)
plt.figure(figsize=(4,6))
sns.barplot(data=df2, y='industry', x='% change', hue='employee', order=order.index)
plt.legend(loc='upper left', bbox_to_anchor=(-0.5,-0.1))

In [ ]:
order = df2.groupby('industry')[df2.columns[3]].max().sort_values(ascending=False).head(20)
plt.figure(figsize=(4,6))
sns.barplot(data=df2, y='industry', x=df2.columns[3], hue='employee', order=order.index)
plt.legend(loc='upper left', bbox_to_anchor=(-0.5,-0.1))

In [ ]:
# sns.set_palette('Dark2')
sns.color_palette('tab10_r')
# sns.color_palette()

# WIP

In [ ]:
url = 'https://download.bls.gov/pub/time.series/le/le.series'
df = pd.read_csv(url, sep='\t')

url = 'https://download.bls.gov/pub/time.series/le/le.data.0.Current'
df1 = pd.read_csv(url, sep='\t')

In [ ]:
# earn_code 01
# fips_code 00
# indy_code 0000

In [ ]:
cond1 = df.earn_code == 1
cond2 = df.fips_code == 0
cond3 = df.indy_code == 0

df[cond1 & cond2 & cond3].iloc[:,0:4]

In [ ]:
a = df1.columns[0]
v = df1.columns[3]

display(a, v)

In [ ]:
data = df1[df1[a] == df1[a][0]]
data[v] = data[v].str.lstrip(' ').astype(float)

plt.figure(figsize=(8,4))
sns.lineplot(data = data[data.year > 2016], x=data[data.year > 2016].index, y=v);

In [ ]:
data[data.year >= 2019]